<a href="https://colab.research.google.com/github/erica-mccormick/waterpyk/blob/main/waterpyk_tutorialL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Basic tutorial for waterpyk**

waterpyk is a Python module for extracting and manipulating timeseries data by leveraging the Google Earth Engine (GEE) and USGS APIs. 

All functionality is described *in detail* in the docs: https://waterpyk.readthedocs.io/en/latest/

Last updated by Erica McCormick | May 31, 2022

### Installation

waterpyk requires geopandas and ee, so first install those if you are using colab.

If you are not using colab, see [here](https://developers.google.com/earth-engine/guides/python_install) to save GEE authentication information in your environment.

In [1]:
pip install geopandas -q

In [ ]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
ee.Authenticate()
ee.Initialize()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

### Import waterpyk package from PyPi

In [ ]:
!pip install -i https://test.pypi.org/simple/ waterpyk==1.1.7

#**Part 1: USING THE STUDYAREA CLASS**

### This is the primary way of interacting with waterpyk if you just want all of the 'normal' data for a site or watershed. First, lets import the StudyArea class from the main module of waterpyk:

In [5]:
from waterpyk.main import StudyArea

### Supply a lat/long or USGS watershed ID and specify that you want 'all' of the default data to be extracted. Now you're done!

### Let's test it on a site called Rivendell. We can see some information about what we've extracted by using the `describe()` function.

In [ ]:
rivendell = StudyArea(layers = 'all', coords = [39.7273, -123.6433])

### Now the rivendell object contains attributes with all of the extracted data, stored in dataframes, such as:
* **daily_df_wide**: daily timeseries (including the deficit and columns for cumulative wateryear values)
* **wateryear_totals**: wateryear summed timeseries
* **stats**: statistics such as smax, landcover type, and elevation

In [ ]:
rivendell.daily_df_wide.head()

In [ ]:
rivendell.wateryear_totals.head()

In [ ]:
rivendell.smax

In [ ]:
rivendell.stats

### We can also make plots without doing any extra work.
### We'll change the dpi here to make it easier to see. The default is 300.
### Try out all of the kinds on your own:
* **kind = 'timeseries'**
* **kind = 'spearman'**: correlation between P and summer ET
* **kind = 'RWS'**: timeseries
* **kind = 'wateryear'**: timeseries of total P, ET, Q, etc.
* **kind = 'distribution'**: distribution of P relative to Smax

In [ ]:
rivendell.plot(kind = 'timeseries', dpi = 100);

In [ ]:
rivendell.plot(kind = 'spearman', dpi = 100);

#**PART 2: MORE FUNCTIONALITY**

### Part 1 is all you need to use waterpyk. However, all of the functionality of StudyArea is also available independently.

###**NOTE: ALL OF THE CODE BELOW (AND MORE) ALREADY HAPPENS WHEN YOU USE STUDYAREA!!!**

### **READ THE DOCS FOR MORE INFO:** https://waterpyk.readthedocs.io/en/latest/

### There are also may optional arguments you can pass to change the above behavior of extractions and plotting. These are described in the docs. **This code is not exhaustive - there is much more you can do!**

###**watershed module**

In [26]:
from waterpyk import watershed

gauge_id = 11475560

# Extract gee and geopandas geometry
gee_geometry, geopandas_geometry = watershed.extract_geometry(gauge_id)

# Extract streamflow dataframe
streamflow = watershed.extract_streamflow(gauge_id)

# Extract metadata
metadata = watershed.extract_metadata(gauge_id)


### Lets see it:
print('Streamflow df:\n', streamflow.head(), '\n')
print('Metadata:\n', metadata)


Streamflow data is being retrieved from: https://labs.waterdata.usgs.gov/api/nldi/linked-data/nwissite/USGS-11475560/navigation/UM/flowlines?f=json&distance=1000 

Streamflow df:
    Q_cfs       date      Q_m3day       Q_m      Q_mm
0   0.66 1980-10-04  1614.953271  0.000096  0.096327
1   0.66 1980-10-05  1614.953271  0.000096  0.096327
2   0.66 1980-10-06  1614.953271  0.000096  0.096327
3   0.66 1980-10-07  1614.953271  0.000096  0.096327
4   0.66 1980-10-08  1614.953271  0.000096  0.096327 

Metadata:
 (['Elder C Nr Branscomb Ca'], 'USGS Basin (11475560) imported at Elder C Nr Branscomb CaCRS: epsg:4326')


###**gee module**

In [30]:
from waterpyk import gee

gee_feature = gee_geometry # from watershed method, above
kind = 'watershed'

# Given the information on a single asset, extract that with extract_basic()
# If the asset is an image, specify relative_date = 'image'
# If the asset should be extracted for a single time period, specify relative_date = 'first' or 'most_recent'
# Otherwise, give start_date and end_date to get a timeseries.

### Example: PML
asset_id = 'projects/pml_evapotranspiration/PML/OUTPUT/PML_V2_8day_v016'
start_date = '2003-10-01'
end_date = '2004-10-01'
scale = 500
relative_date = None
bands = ['Es', 'Ec','Ei']
bands_to_scale = None
scaling_factor = 1
reducer_type = None

print('NOTICE HOW THIS iS NOT CLEANED OR FORMATTED? USE STUDYAREA TO GET BETTER DATA, FASTER\n')
pml = df = gee.extract_basic(gee_feature, kind, asset_id, scale, bands, start_date, end_date, relative_date, bands_to_scale, scaling_factor, reducer_type)
pml.head()


NOTICE HOW THIS iS NOT CLEANED OR FORMATTED? USE STUDYAREA TO GET BETTER DATA, FASTER

	Original timestep of 8 day(s) was interpolated to daily.


,variable,value,date,band,value_raw
0,2003-10-08_Ec,163.873696,2003-10-08,Ec,163.873696
1,NaN,160.859257,2003-10-09,Ec,NaN
2,NaN,157.844818,2003-10-10,Ec,NaN
3,NaN,154.830380,2003-10-11,Ec,NaN
4,NaN,151.815941,2003-10-12,Ec,NaN
